In [1]:
import numpy as np
from sklearn.datasets import load_iris
from pprint import pprint
from sklearn.model_selection import train_test_split
import pandas as pd


In [2]:


# Definición de la función para calcular la entropía
def entropy(y):
    unique, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# Definición de la función para dividir los datos
def split_data(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] <= threshold
    right_mask = ~left_mask
    X_left, y_left = X[left_mask], y[left_mask]
    X_right, y_right = X[right_mask], y[right_mask]
    return X_left, y_left, X_right, y_right

# Definición de la clase del nodo del árbol de decisión
class Node:
    def __init__(self, depth=0, max_depth=None):
        self.depth = depth
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None

    def fit(self, X, y):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        self.value = unique_classes[np.argmax(class_counts)]
        if self.depth == self.max_depth or len(unique_classes) == 1: # Se pregunta las dos condiciones de paradas, sin divisiones y profundidad máxima
            return

        num_features = X.shape[1] # cantidad de Features
        best_info_gain = -1 # métrica
        
        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = split_data(X, y, feature_index, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                info_gain = entropy(y) - ((len(y_left) / len(y)) * entropy(y_left) + (len(y_right) / len(y)) * entropy(y_right))

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    self.feature_index = feature_index
                    self.threshold = threshold

        if self.feature_index is not None:
            X_left, y_left, X_right, y_right = split_data(X, y, self.feature_index, self.threshold)
            self.left = Node(depth=self.depth + 1, max_depth=self.max_depth)
            self.left.fit(X_left, y_left)
            self.right = Node(depth=self.depth + 1, max_depth=self.max_depth)
            self.right.fit(X_right, y_right)

    def predict(self, X):
        if self.feature_index is None:
            return self.value
        if X[self.feature_index] <= self.threshold:
            return self.left.predict(X)
        else:
            return self.right.predict(X)



In [47]:
# Leer los csv files

df1 = pd.read_table("DataSetNotas/notas1.csv", delimiter =",")
df1 = df1[['Examen 1 25.0%', 'Examen 2 25.0%', 'Quiz1', 'Total: ']]
df1.columns = ['E1', 'E2', 'Q1', 'T']
df1 = df1.fillna(0)

df2 = pd.read_table("DataSetNotas/notas2.csv", delimiter =";")
df2 = df2[['E1 25.00%', 'E2 25.00%', 'Q1 3.00%', 'Total: ']]
df2.columns = ['E1', 'E2', 'Q1', 'T']
df2 = df2.fillna(0)

df3 = pd.read_table("DataSetNotas/notas3.csv", delimiter =",")
df3 = df3[['Examen 1 25.00%', 'Examen 2 25.00%', 'Quiz 1', 'Total: ']]
df3.columns = ['E1', 'E2', 'Q1', 'T']
df3 = df3.fillna(0)

df4 = pd.read_table("DataSetNotas/notas4.csv", delimiter =",")
df4 = df4[['Examen 1 25.00%', 'Examen 2 25%', 'Quiz 1', 'Total: ']]
df4.columns = ['E1', 'E2', 'Q1', 'T']
df4 = df4.fillna(0)

df5 = pd.read_table("DataSetNotas/notas5.csv", delimiter =";")
df5 = df5[['Examen 1 25%', 'Examen 2 25.00%', 'Q1 3%', 'Total: ']]
df5 = df5.replace(' ', 0)
df5.columns = ['E1', 'E2', 'Q1', 'T']

frames = [df1, df2, df3, df4, df5]
allData = pd.concat(frames, ignore_index=True)
allData = allData.apply(pd.to_numeric, errors='coerce')
allData.fillna(0)

allData.head(125)

,E1,E2,Q1,T
0,57.4,75.6,93.3,70.50
1,100.0,96.3,86.7,95.80
2,19.6,46.7,26.7,44.80
3,34.5,34.2,36.7,28.40
4,89.5,87.7,93.3,91.40
...,...,...,...,...
119,45.0,95.0,35.0,67.55
120,100.0,75.0,80.0,83.90
121,10.0,0.0,35.0,7.55
122,15.0,0.0,10.0,4.05


In [3]:
iris = load_iris()
x = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)

In [4]:
# Creación y entrenamiento del árbol de decisión
max_depth = 3
tree = Node(max_depth=max_depth)
tree.fit(X_train, y_train)

# Realización de predicciones en el conjunto de prueba
y_pred = [tree.predict(x) for x in X_test]

# Evaluación del modelo
accuracy = np.mean(y_pred == y_test)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.96
